In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import re 
import string

In [2]:
askmen = pd.read_csv('../Data/askmen_df.csv')

In [3]:
askwomen = pd.read_csv('../Data/askwomen_df.csv')

In [4]:
df = askmen.append(askwomen, ignore_index=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [5]:
df.shape

(5900, 69)

In [6]:
df['selftext']

0                                               [removed]
1       So over the past month or so, I’ve gotten pret...
2       I just became curious because I saw this porn ...
3                                               [removed]
4                                               [removed]
                              ...                        
5895                                            [removed]
5896                                            [removed]
5897    So my husband sometimes uses my phone to get o...
5898                                            [removed]
5899                                            [removed]
Name: selftext, Length: 5900, dtype: object

In [7]:
no_removed = df[df['selftext'] != '[removed]']

In [8]:
no_removed.shape

(3272, 69)

In [9]:
df['subreddit'].value_counts()

AskWomen    2964
AskMen      2936
Name: subreddit, dtype: int64

In [10]:
no_removed = no_removed[['title', 'selftext', 'subreddit']]

In [11]:
no_removed.shape

(3272, 3)

In [12]:
no_removed['text'] = no_removed['title'] + no_removed['selftext'].fillna('')

In [13]:
no_na = no_removed.dropna()

In [14]:
no_na['subreddit'].value_counts()

AskMen      1003
AskWomen     593
Name: subreddit, dtype: int64

In [15]:
tokenizer = RegexpTokenizer('\s+', gaps=True)

In [16]:
def clean_text(text):
    text = text.lower()
    regex = re.compile("[^a-zA-Z0-9!? ]")
    text = regex.sub('', text)
    return text

In [19]:
df['text'] = df['title'] + df['selftext'].fillna('')

In [20]:
df.head()

,all_awardings,allow_live_comments,author,author_cakeday,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,...,subreddit_id,subreddit_subscribers,subreddit_type,thumbnail,title,total_awards_received,url,whitelist_status,wls,text
0,[],False,RonReag77,NaN,NaN,NaN,[],NaN,NaN,NaN,...,t5_2s30g,1636380,public,self,I get rejected a lot for my looks. What advice...,0.0,https://www.reddit.com/r/AskMen/comments/ewjoj...,all_ads,6,I get rejected a lot for my looks. What advice...
1,[],False,dubfegnezz,NaN,NaN,NaN,[],NaN,NaN,NaN,...,t5_2s30g,1636371,public,self,Any insight on how to handle a coworker trying...,0.0,https://www.reddit.com/r/AskMen/comments/ewjmx...,all_ads,6,Any insight on how to handle a coworker trying...
2,[],False,AdamtheFirstSinner,NaN,#83acfa,male,"[{'e': 'text', 't': 'Professional Smartass'}]",ea22c17e-cd16-11e1-845d-12313b0ce1e2,Professional Smartass,light,...,t5_2s30g,1636365,public,self,"Women of reddit, what are you really getting o...",0.0,https://www.reddit.com/r/AskMen/comments/ewjmd...,all_ads,6,"Women of reddit, what are you really getting o..."
3,[],False,AgentSuperfly17,NaN,NaN,NaN,[],NaN,NaN,NaN,...,t5_2s30g,1636363,public,self,Resistance Band Fitness Advice Please Help,0.0,https://www.reddit.com/r/AskMen/comments/ewjlw...,all_ads,6,Resistance Band Fitness Advice Please Help[rem...
4,[],False,dubfegnezz,NaN,NaN,NaN,[],NaN,NaN,NaN,...,t5_2s30g,1636359,public,self,A coworker is trying to set me up with another...,0.0,https://www.reddit.com/r/AskMen/comments/ewjlk...,all_ads,6,A coworker is trying to set me up with another...


In [21]:
df['text_clean'] = df['text'].apply(lambda x:clean_text(x))

In [22]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in tokenizer.tokenize(text)]

df['text_clean_lemmatized'] = df['text_clean'].apply(lemmatize_text)

In [23]:
seperator = ' '

df['text_clean_lemmatized'] = df['text_clean_lemmatized'].apply(lambda x: seperator.join(x))

In [24]:
df['text_lemmatized'] = df['text'].apply(lemmatize_text)

In [25]:
seperator = ' '

df['text_lemmatized'] = df['text_lemmatized'].apply(lambda x: seperator.join(x))

In [27]:
no_removed['subreddit'] = no_removed['subreddit'].map(lambda x:1 if x == 'AskWomen' else 0)

In [28]:
no_removed.to_csv('../Data/no_removed.csv', index = False)

In [30]:
askmen

,all_awardings,allow_live_comments,author,author_cakeday,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,...,subreddit,subreddit_id,subreddit_subscribers,subreddit_type,thumbnail,title,total_awards_received,url,whitelist_status,wls
0,[],False,RonReag77,NaN,NaN,NaN,[],NaN,NaN,NaN,...,AskMen,t5_2s30g,1636380,public,self,I get rejected a lot for my looks. What advice...,0,https://www.reddit.com/r/AskMen/comments/ewjoj...,all_ads,6
1,[],False,dubfegnezz,NaN,NaN,NaN,[],NaN,NaN,NaN,...,AskMen,t5_2s30g,1636371,public,self,Any insight on how to handle a coworker trying...,0,https://www.reddit.com/r/AskMen/comments/ewjmx...,all_ads,6
2,[],False,AdamtheFirstSinner,NaN,#83acfa,male,"[{'e': 'text', 't': 'Professional Smartass'}]",ea22c17e-cd16-11e1-845d-12313b0ce1e2,Professional Smartass,light,...,AskMen,t5_2s30g,1636365,public,self,"Women of reddit, what are you really getting o...",0,https://www.reddit.com/r/AskMen/comments/ewjmd...,all_ads,6
3,[],False,AgentSuperfly17,NaN,NaN,NaN,[],NaN,NaN,NaN,...,AskMen,t5_2s30g,1636363,public,self,Resistance Band Fitness Advice Please Help,0,https://www.reddit.com/r/AskMen/comments/ewjlw...,all_ads,6
4,[],False,dubfegnezz,NaN,NaN,NaN,[],NaN,NaN,NaN,...,AskMen,t5_2s30g,1636359,public,self,A coworker is trying to set me up with another...,0,https://www.reddit.com/r/AskMen/comments/ewjlk...,all_ads,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931,[],False,razaya,NaN,NaN,NaN,[],NaN,NaN,NaN,...,AskMen,t5_2s30g,1626134,public,self,What is the most chicken you have eaten in one...,0,https://www.reddit.com/r/AskMen/comments/eucq1...,all_ads,6
2932,[],False,SholoGrim,NaN,NaN,NaN,[],NaN,NaN,NaN,...,AskMen,t5_2s30g,1626128,public,self,Homecoming Drama,0,https://www.reddit.com/r/AskMen/comments/eucnq...,all_ads,6
2933,[],False,Hhhnmmggaa,NaN,NaN,NaN,[],NaN,NaN,NaN,...,AskMen,t5_2s30g,1626119,public,self,Do guys always say “this is the best blowjob i...,0,https://www.reddit.com/r/AskMen/comments/eucm3...,all_ads,6
2934,[],False,TotalArea,NaN,#83acfa,male,"[{'e': 'text', 't': 'Male'}]",ea22c17e-cd16-11e1-845d-12313b0ce1e2,Male,light,...,AskMen,t5_2s30g,1626115,public,self,"RIP Kobe Bryant, how will you remember Kobe Br...",0,https://www.reddit.com/r/AskMen/comments/euclp...,all_ads,6


In [31]:
men = askmen[['title', 'selftext', 'subreddit', 'permalink', 'created_utc']]

In [32]:
women = askwomen[['title', 'selftext', 'subreddit','permalink', 'created_utc' ]]

In [33]:
df = women.append(men, ignore_index=True)

In [36]:
df.to_csv('../Data/both_5cols.csv')

In [37]:
titles_clean = pd.DataFrame(df['title'].apply(lambda x:clean_text(x))) 

In [115]:
titles_clean.head(10)

,title,subreddit
0,why is makeup so important? why cant some girl...,AskWomen
1,do you ever find the close male friendshipbrom...,AskWomen
2,why do some women not like being called babe a...,AskWomen
3,peeing instead of having an orgasm help!,AskWomen
4,isshould gender be assigned,AskWomen
5,sneezing too hard feels like tearing abdomen,AskWomen
6,does his matter to you? have you ever been di...,AskWomen
7,whatcha think about males with piercings?,AskWomen
8,women of reddit should guys shave the bush?,AskWomen
9,what is the smallest thing youve broken up wit...,AskWomen


In [39]:
titles_clean['subreddit'] = df['subreddit']

In [40]:
df['subreddit'] = df['subreddit'].map(lambda x:1 if x == 'AskWomen' else 0)

In [42]:
(df['selftext'] == "[removed]").value_counts()

False    3272
True     2628
Name: selftext, dtype: int64

In [43]:
df['selftext'][ df['selftext'] == "[removed]"] = ''

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
(df['selftext'] == "").value_counts()

False    3272
True     2628
Name: selftext, dtype: int64

In [45]:
(women['selftext'] == "[removed]").value_counts(normalize = True)

False    0.525641
True     0.474359
Name: selftext, dtype: float64

In [46]:
(men['selftext'] == "[removed]").value_counts(normalize = True)

False    0.583787
True     0.416213
Name: selftext, dtype: float64

In [47]:
df['text'] = df['title'].fillna('').map(str) + df['selftext'].fillna('').map(str)

In [48]:
df[['text']]

,text
0,Why is makeup so important? Why can't some gir...
1,Do you ever find the close male friendship/bro...
2,"Why do some women not like being called babe, ..."
3,Peeing Instead of Having an Orgasm Help!Hi eve...
4,Is/should gender be assigned
...,...
5895,What is the most chicken you have eaten in one...
5896,Homecoming Drama
5897,Do guys always say “this is the best blowjob i...
5898,"RIP Kobe Bryant, how will you remember Kobe Br..."


In [49]:
df['permalink'][2940]

'/r/AskWomen/comments/90v48m/do_straight_women_get_upset_or_angry_if_another/'

In [50]:
(men['selftext'] == "[removed]")

0        True
1       False
2       False
3        True
4        True
        ...  
2931    False
2932     True
2933     True
2934    False
2935     True
Name: selftext, Length: 2936, dtype: bool

In [51]:
men.iloc[(1)]['permalink']

'/r/AskMen/comments/ewjmxx/any_insight_on_how_to_handle_a_coworker_trying_to/'

In [52]:
tokenizer = RegexpTokenizer('\s+', gaps=True)

In [53]:
cvec = CountVectorizer(min_df=2, stop_words='english', strip_accents= 'unicode')

In [54]:
y = df['subreddit']

In [55]:
x = titles_clean['title']

In [56]:
x_train, x_test, y_train, y_test = train_test_split(x, y) 

In [57]:
cvec.fit(x_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=2,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents='unicode', token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [58]:
x_train_cvec = cvec.transform(x_train)
x_test_cvec = cvec.transform(x_test)

In [59]:
x_train_cvec

<4425x2364 sparse matrix of type '<class 'numpy.int64'>'
	with 20928 stored elements in Compressed Sparse Row format>

In [60]:
titles = pd.DataFrame(x_train_cvec.todense(), columns = cvec.get_feature_names())

In [61]:
titles['subreddit'] = df['subreddit']

In [63]:
titles[titles['subreddit'] == 1].sum().sort_values(ascending = False).head(20)

subreddit       2964
men              298
women            283
like             173
did              172
feel             157
guys             156
whats            140
guy              138
girl             138
reddit           137
sex              126
does             120
think            109
friends           99
woman             97
make              97
just              92
time              90
relationship      89
dtype: int64

In [116]:
titles[titles['subreddit'] == 0].sum().sort_values(ascending = False).head(10)

women     135
men       130
like      119
whats      92
guys       79
did        77
does       74
guy        71
reddit     67
feel       63
dtype: int64

In [65]:
vocab_men = (titles[titles['subreddit'] == 0].sum()) > 0 

In [66]:
vocab_women = (titles[titles['subreddit'] == 1].sum()) > 0 

In [71]:
titles.sum().sort_values(ascending=False)

subreddit    2964
men           428
women         418
like          292
did           249
             ... 
gifting         2
studies         2
ghosted         2
gf24            2
knocked         2
Length: 2364, dtype: int64

In [72]:
vectorizer2 = TfidfVectorizer(stop_words='english', min_df=2, max_df=1000, )

In [73]:
x_train_tf = vectorizer2.fit_transform(x_train)
x_test_tf = vectorizer2.transform(x_test)

In [74]:
titles_tf = pd.DataFrame(x_train_tf.todense(), columns=vectorizer2.get_feature_names())

In [76]:
stops = titles_tf.sum().sort_values(ascending = False).head(20)

In [77]:
lr = LogisticRegression()

In [78]:
lr.fit(x_train_cvec, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [79]:
lr.score(x_train_cvec, y_train)

0.8366101694915254

In [80]:
lr.score(x_test_cvec, y_test)

0.6311864406779661

In [81]:
lr.fit(x_train_tf, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [82]:
lr.score(x_train_tf, y_train)

0.8024858757062147

In [83]:
lr.score(x_test_tf, y_test)

0.6393220338983051

In [84]:
x = df['text']
y = df['subreddit']

In [85]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [86]:
cvec.fit(x_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=2,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents='unicode', token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [87]:
x_train_cvec = cvec.transform(x_train)
x_test_cvec = cvec.transform(x_test)

In [88]:
lr.fit(x_train_cvec, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [89]:
lr.score(x_train_cvec, y_train)

0.8842937853107344

In [90]:
lr.score(x_test_cvec, y_test)

0.6596610169491526

In [91]:
text = pd.DataFrame(x_train_cvec.todense(), columns = cvec.get_feature_names())

In [92]:
text

,000,10,100,100k,10k,10th,11,12,13,14,...,youre,youth,youtu,youtube,youtubers,yr,zero,zodiac,zone,zoned
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4421,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4422,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4423,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
def clean_text(text):
    text = text.lower()
    regex = re.compile("[^a-zA-Z0-9!? ]")
    text = regex.sub('', text)
    return text

text_clean = pd.DataFrame(df['text'].apply(lambda x:clean_text(x))) 

In [94]:
text_clean['titles'] = pd.DataFrame(df['title'].apply(lambda x:clean_text(x))) 

In [95]:
text_clean

,text,titles
0,why is makeup so important? why cant some girl...,why is makeup so important? why cant some girl...
1,do you ever find the close male friendshipbrom...,do you ever find the close male friendshipbrom...
2,why do some women not like being called babe a...,why do some women not like being called babe a...
3,peeing instead of having an orgasm help!hi eve...,peeing instead of having an orgasm help!
4,isshould gender be assigned,isshould gender be assigned
...,...,...
5895,what is the most chicken you have eaten in one...,what is the most chicken you have eaten in one...
5896,homecoming drama,homecoming drama
5897,do guys always say this is the best blowjob iv...,do guys always say this is the best blowjob iv...
5898,rip kobe bryant how will you remember kobe bry...,rip kobe bryant how will you remember kobe bry...


In [96]:
lemmatizer = WordNetLemmatizer()

In [97]:
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in tokenizer.tokenize(text)]

In [98]:
text_clean['lemmatized'] = text_clean.text.apply(lemmatize_text)

In [99]:
seperator = ' '

text_clean['lemmatized'] = text_clean['lemmatized'].apply(lambda x: seperator.join(x))

In [100]:
seperator = ' '

text_clean['titles_lemmatized'] = text_clean.titles.apply(lemmatize_text)

In [101]:
seperator = ' '

text_clean['titles_lemmatized'] = text_clean.titles_lemmatized.apply(lambda x: seperator.join(x))

In [102]:
text_clean

,text,titles,lemmatized,titles_lemmatized
0,why is makeup so important? why cant some girl...,why is makeup so important? why cant some girl...,why is makeup so important? why cant some girl...,why is makeup so important? why cant some girl...
1,do you ever find the close male friendshipbrom...,do you ever find the close male friendshipbrom...,do you ever find the close male friendshipbrom...,do you ever find the close male friendshipbrom...
2,why do some women not like being called babe a...,why do some women not like being called babe a...,why do some woman not like being called babe a...,why do some woman not like being called babe a...
3,peeing instead of having an orgasm help!hi eve...,peeing instead of having an orgasm help!,peeing instead of having an orgasm help!hi eve...,peeing instead of having an orgasm help!
4,isshould gender be assigned,isshould gender be assigned,isshould gender be assigned,isshould gender be assigned
...,...,...,...,...
5895,what is the most chicken you have eaten in one...,what is the most chicken you have eaten in one...,what is the most chicken you have eaten in one...,what is the most chicken you have eaten in one...
5896,homecoming drama,homecoming drama,homecoming drama,homecoming drama
5897,do guys always say this is the best blowjob iv...,do guys always say this is the best blowjob iv...,do guy always say this is the best blowjob ive...,do guy always say this is the best blowjob ive...
5898,rip kobe bryant how will you remember kobe bry...,rip kobe bryant how will you remember kobe bry...,rip kobe bryant how will you remember kobe bry...,rip kobe bryant how will you remember kobe bry...


In [104]:
text_clean.to_csv('../Data/text_clean.csv', index = False)

In [105]:
text_clean['subreddit'] = df['subreddit']

In [106]:
x = text_clean['lemmatized']
y = text_clean['subreddit']

x_train, x_test, y_train, y_test = train_test_split(x, y)

In [107]:
cvec = CountVectorizer(min_df=2, stop_words='english', max_df= 1000, ngram_range= (1, 2))

In [108]:
cvec.fit(x_train)

x_train_cvec = cvec.transform(x_train)
x_test_cvec = cvec.transform(x_test)

In [109]:
x_train_cvec 

<4425x9978 sparse matrix of type '<class 'numpy.int64'>'
	with 71585 stored elements in Compressed Sparse Row format>

In [110]:
pd.DataFrame(x_train_cvec.todense(), columns=cvec.get_feature_names())

,10,10 day,10 year,100,1000,1015lbs,1015lbs lift,1020,10th,11,...,youve received,youve seen,youve teach,youve used,yr,zero,zero experience,zodiac,zone,zoned
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4421,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4422,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4423,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
lr.fit(x_train_cvec, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [112]:
lr.score(x_train_cvec, y_train)

0.9380790960451978

In [113]:
lr.score(x_test_cvec, y_test)

0.6610169491525424

In [114]:
#is there a difference in average post length